In [1]:
import pandas as pd
import numpy as np
import requests
import random
import math
import matplotlib.pyplot as plt
from scipy import stats
from scipy import stats, special
from sklearn import model_selection, metrics, linear_model, datasets, feature_selection
from lxml import html

In [2]:
PGA_base_url = "https://statdata.pgatour.com/r/" 

#Check PGA for current tournament

PGA_current= "https://statdata.pgatour.com/r/current/message.json"
r = requests.get(PGA_current)
r_json = r.json()
current_id = r_json['tid']
current_url = "https://statdata.pgatour.com/r/{0}".format(current_id)

#Current tournament's JSON

PGA_current_url = "{0}/leaderboard-v2mini.json".format(current_url)

r = requests.get(PGA_current_url)
r_json = r.json()
current_tournament = r_json["debug"]["tournament_in_schedule_file_name"]
print(current_tournament)
print(PGA_current_url)

PGA Championship
https://statdata.pgatour.com/r/033/leaderboard-v2mini.json


In [3]:
team1 = "RBosshard"
team1_list = {"Jordan Spieth", "Justin Rose", "Jon Rahm", "Tiger Woods", "Louis Oosthuizen"}

team2 = "JKuo"
team2_list = {"Dustin Johnson", "Tommy Fleetwood", "Brooks Koepka", "Marc Leishman", "Tony Finau"}

team3 = "JLo"
team3_list = {"Rory McIlroy", "Rickie Fowler", "Henrik Stenson", "Justin Thomas", "Charley Hoffman"}


teams = {team1 : team1_list, team2 : team2_list, team3 : team3_list}

In [4]:
PGA_base_url = "https://statdata.pgatour.com/r/" 

#Check PGA for current tournament

PGA_current= "https://statdata.pgatour.com/r/current/message.json"
r = requests.get(PGA_current)
r_json = r.json()
current_id = r_json['tid']
current_url = "https://statdata.pgatour.com/r/{0}".format(current_id)

#Current tournament's JSON

PGA_current_url = "https://statdata.pgatour.com/r/100/leaderboard-v2mini.json"
r = requests.get(PGA_current_url)
r_json = r.json()
current_tournament = r_json["debug"]["tournament_in_schedule_file_name"]

#Create Scoreboard

field = 156
score = r_json["leaderboard"]["players"]

team_list = []

for team in teams.keys():
    players = teams[team]
    for player in players:
        name = player.split()
        for i in range(0, field):
            try:
                if name[0] == score[i]["player_bio"]["first_name"]:
                    if name[1] == score[i]["player_bio"]["last_name"]:
                        row_dict = {"Team": team,
                                "Player": player,
                                'Today': score[i]["today"],
                                'Total' : score[i]["total"],
                                'Thru': score[i]["thru"]}
                        team_list.append(row_dict)

            except:
                print(i)

df = pd.DataFrame(team_list)
df = df.set_index("Team")
df = df[["Player","Today","Total","Thru"]]


#create team score

score_list = []
for team in teams.keys():
    score = 0
    count = -1 
    for i in list(df.index):
        count = count + 1
        if i == team:
            score = score + df.iloc[count][1]
    score_dict = {"Team": team, "Score": score}
    score_list.append(score_dict)        
    
df_score = pd.DataFrame(score_list)
df_score = df_score.set_index("Team")

df["Score"] = df_score

#clean scores

for i in range(0,len(df.index)):
    if i % len(list(teams[team1])) != 0:
        df.iloc[i, df.columns.get_loc('Score')] = ""
        
#drop the worst score

    else:
        df.iloc[i, df.columns.get_loc('Score')] = df.iloc[i][4] - df.iloc[i:i+len(list(teams[team1]))]["Today"].max()


df

,Player,Today,Total,Thru,Score
Team,,,,,
RBosshard,Louis Oosthuizen,2.0,0,18.0,NaN
RBosshard,Justin Rose,-2.0,-6,18.0,
RBosshard,Tiger Woods,0.0,-5,18.0,
RBosshard,Jon Rahm,NaN,5,NaN,
RBosshard,Jordan Spieth,5.0,-4,18.0,
JKuo,Marc Leishman,5.0,5,18.0,NaN
JKuo,Tony Finau,0.0,-4,18.0,
JKuo,Tommy Fleetwood,2.0,-3,18.0,
JKuo,Brooks Koepka,-1.0,2,18.0,


In [5]:
#Current Season Champion List

PGA_base_url = "https://statdata.pgatour.com/r/" 

#Check PGA for current tournament

PGA_current= "https://statdata.pgatour.com/r/current/message.json"
r = requests.get(PGA_current)
r_json = r.json()
current_id = r_json['tid']

tourn_id = ['002',
 '003',
 '004',
 '005',
 '006',
 '007',
 '009',
 '010',
 '011',
 '012',
 '014',
 '016',
 '018',
 '019',
 '020',
 '021',
 '023',
 '025',
 '026',
 '034',
 '041',
 '471',
 '473',
 '475',
 '480',
 '490',
 '030',
 '100',
 '518']

champion_list = []

for i in tourn_id:
    
    try:

        current_url = "https://statdata.pgatour.com/r/{0}".format(i)

        #Current tournament's JSON

        PGA_current_url = "{0}/leaderboard-v2mini.json".format(current_url)

        r = requests.get(PGA_current_url)
        r_json = r.json()
        current_tournament = r_json["debug"]["tournament_in_schedule_file_name"]
        champion = r_json["leaderboard"]["players"][0]["player_bio"]["last_name"]
        date = r_json["leaderboard"]["end_date"]
        second = r_json["leaderboard"]["players"][1]["player_bio"]["last_name"]
        third = r_json["leaderboard"]["players"][2]["player_bio"]["last_name"]
        fourth = r_json["leaderboard"]["players"][3]["player_bio"]["last_name"]
        fifth = r_json["leaderboard"]["players"][4]["player_bio"]["last_name"]
        
        row_dict = {"Tournament" : current_tournament, "Champion" : champion, "Date"  : date, "2nd" : second,
                   "3rd" : third, "4th" : fourth, "5th" : fifth}
        
        champion_list.append(row_dict)
        
    except:
        
        print(PGA_current_url)
        
        
        

df = pd.DataFrame(champion_list)
df = df.set_index("Date")
df = df[["Tournament","Champion", "2nd", "3rd", "4th", "5th"]]
df=df.sort_index()

df

,Tournament,Champion,2nd,3rd,4th,5th
Date,,,,,,
2018-01-07,Sentry Tournament of Champions,Johnson,Rahm,Harman,Matsuyama,Perez
2018-01-14,Sony Open in Hawaii,Kizzire,Hahn,Hoge,Simpson,Stuard
2018-01-21,CareerBuilder Challenge,Rahm,Landry,Huh,Hadwin,Piller
2018-01-28,Farmers Insurance Open,Day,Noren,Palmer,Holmes,Bradley
2018-02-04,Waste Management Phoenix Open,Woodland,Reavie,Schniederjans,Steele,Hadley
2018-02-11,AT&T Pebble Beach Pro-Am,"Potter, Jr.",Mickelson,Reavie,Day,Johnson
2018-02-18,Genesis Open,Watson,Na,Finau,Stallings,Cantlay
2018-02-25,The Honda Classic,Thomas,List,Noren,Fleetwood,An
2018-03-04,World Golf Championships-Mexico Championship,Mickelson,Thomas,Cabrera Bello,Hatton,Aphibarnrat


In [6]:
#Current PGA Rankings

page = requests.get('http://www.owgr.com/ranking')
tree = html.fromstring(page.content)

#This will create a list of golfers:
golfers = tree.xpath('//table//td/a/text()')

b = 1
golfer_list = []

for a in golfers:
    
    c = a.split()
    row_dict = {"First Name" : c[0], "Last Name" : c[1],  "Ranking" : b}        
    golfer_list.append(row_dict)
    b = b + 1
    
df2 = pd.DataFrame(golfer_list)
df2 = df2.set_index("Ranking")

df2

,First Name,Last Name
Ranking,,
1,Dustin,Johnson
2,Justin,Thomas
3,Justin,Rose
4,Brooks,Koepka
5,Rory,McIlroy
6,Francesco,Molinari
7,Jon,Rahm
8,Jordan,Spieth
9,Rickie,Fowler


In [7]:
top5_list=[]
top5 = 0
ranking = "+100"
player=np.unique(df[["Champion", "2nd", "3rd", "4th", "5th"]].values)
for i in player:
    top5=0
    for a in range(len(df.index)):
        if df.iloc[a]["Champion"] == i or df.iloc[a]["2nd"] == i or df.iloc[a]["3rd"] == i or df.iloc[a]["4th"] == i or df.iloc[a]["5th"] == i:
            top5 = top5 + 1
            date = df.index[a]
    try:
        a = df2.loc[df2['Last Name'] == i]
        ranking = a.index[0]
    except:
        ranking = "+100"
       
    row_dict = {"Player" : i, "Top 5 Finishes" : top5, "Current Ranking" : ranking, "Most Recent Finish" : date}        
    top5_list.append(row_dict)
df1 = pd.DataFrame(top5_list)
df1 = df1.set_index("Player")
df1 = df1.sort_values(['Top 5 Finishes'], ascending=False)
df1 = df1[["Top 5 Finishes", "Current Ranking", "Most Recent Finish"]]

df1

,Top 5 Finishes,Current Ranking,Most Recent Finish
Player,,,
Johnson,5,1,2018-06-17
Day,4,10,2018-05-13
DeChambeau,4,22,2018-06-03
Woods,3,51,2018-07-01
Holmes,3,85,2018-06-24
Mickelson,3,21,2018-05-06
Molinari,3,6,2018-07-22
Na,3,41,2018-07-08
Rahm,3,7,2018-04-08


In [19]:
a = "Spieth"

df1.loc[df1.index == a]

,Top 5 Finishes,Current Ranking,Most Recent Finish
Player,,,
Spieth,2,6,2018-04-08


In [11]:
#Find all tournament IDs

#Check PGA for current tournament

PGA_current= "https://statdata.pgatour.com/r/current/message.json"
r = requests.get(PGA_current)
r_json = r.json()
current_id = r_json['tid']

tourn_id = []

champion_list = []

for i in range(0,500):
    
    if i < 10:
        i = "00{0}".format(i)
    elif i < 100:
        i = "0{0}".format(i)
    else:
        i = "{0}".format(i)
    
    try:
        
        current_url = "https://statdata.pgatour.com/r/{0}".format(i)

        #Current tournament's JSON

        PGA_current_url = "{0}/leaderboard-v2mini.json".format(current_url)

        r = requests.get(PGA_current_url)
        r_json = r.json()
        current_tournament = r_json["debug"]["tournament_in_schedule_file_name"]
        print(current_tournament)
        print(PGA_current_url)
        
        tourn_id.append(i)
        
    except:
        
        print(PGA_current_url, " DOES NOT WORK")

tourn_id



https://statdata.pgatour.com/r/000/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/001/leaderboard-v2mini.json  DOES NOT WORK
CareerBuilder Challenge
https://statdata.pgatour.com/r/002/leaderboard-v2mini.json
Waste Management Phoenix Open
https://statdata.pgatour.com/r/003/leaderboard-v2mini.json
Farmers Insurance Open
https://statdata.pgatour.com/r/004/leaderboard-v2mini.json
AT&T Pebble Beach Pro-Am
https://statdata.pgatour.com/r/005/leaderboard-v2mini.json
Sony Open in Hawaii
https://statdata.pgatour.com/r/006/leaderboard-v2mini.json
Genesis Open
https://statdata.pgatour.com/r/007/leaderboard-v2mini.json
https://statdata.pgatour.com/r/008/leaderboard-v2mini.json  DOES NOT WORK
Arnold Palmer Invitational presented by Mastercard
https://statdata.pgatour.com/r/009/leaderboard-v2mini.json
The Honda Classic
https://statdata.pgatour.com/r/010/leaderboard-v2mini.json
THE PLAYERS Championship
https://statdata.pgatour.com/r/011/leaderboard-v2mini.json
RBC Heritage
https

https://statdata.pgatour.com/r/108/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/109/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/110/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/111/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/112/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/113/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/114/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/115/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/116/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/117/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/118/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/119/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/120/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/121/lea

https://statdata.pgatour.com/r/219/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/220/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/221/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/222/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/223/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/224/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/225/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/226/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/227/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/228/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/229/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/230/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/231/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/232/lea

https://statdata.pgatour.com/r/330/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/331/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/332/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/333/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/334/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/335/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/336/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/337/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/338/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/339/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/340/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/341/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/342/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/343/lea

https://statdata.pgatour.com/r/441/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/442/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/443/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/444/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/445/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/446/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/447/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/448/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/449/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/450/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/451/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/452/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/453/leaderboard-v2mini.json  DOES NOT WORK
https://statdata.pgatour.com/r/454/lea

['002',
 '003',
 '004',
 '005',
 '006',
 '007',
 '009',
 '010',
 '011',
 '012',
 '013',
 '014',
 '016',
 '018',
 '019',
 '020',
 '021',
 '023',
 '025',
 '026',
 '027',
 '028',
 '030',
 '032',
 '033',
 '034',
 '041',
 '047',
 '054',
 '058',
 '060',
 '100',
 '169',
 '457',
 '464',
 '471',
 '472',
 '473',
 '475',
 '476',
 '478',
 '480',
 '483',
 '489',
 '490',
 '493',
 '494']

In [8]:
#The Open July 19, 2018

team1 = "RBosshard"
team1_list = {"Jordan Spieth", "Justin Rose", "Jon Rahm", "Tiger Woods", "Louis Oosthuizen"}

team2 = "JKuo"
team2_list = {"Dustin Johnson", "Tommy Fleetwood", "Brooks Koepka", "Marc Leishman", "Tony Finau"}

team3 = "JLo"
team3_list = {"Rory McIlroy", "Rickie Fowler", "Henrik Stenson", "Justin Thomas", "Charley Hoffman"}


teams = {team1 : team1_list, team2 : team2_list, team3 : team3_list}

In [4]:
#The Open July 19, 2018 Crossley Pool

team1 = "RBosshard"
team1_list = {"Dustin Johnson", "Jordan Spieth", "Marc Leishman", "Webb Simpson", "Tiger Woods", "Tony Finau", "Louis Oosthuizen", "Kevin Na", "Chez Reavie", "Beau Hossler", "Sunghoon Kang", "Martin Kaymer"}

teams = {team1 : team1_list}

In [9]:
#The Open July 19, 2018

PGA_base_url = "https://statdata.pgatour.com/r/" 

#Check PGA for current tournament

PGA_current= "https://statdata.pgatour.com/r/current/message.json"
r = requests.get(PGA_current)
r_json = r.json()
current_id = 100
current_url = "https://statdata.pgatour.com/r/{0}".format(current_id)

#Current tournament's JSON

PGA_current_url = "https://statdata.pgatour.com/r/100/leaderboard-v2mini.json"
r = requests.get(PGA_current_url)
r_json = r.json()
current_tournament = r_json["debug"]["tournament_in_schedule_file_name"]

#Create Scoreboard

field = 156
score = r_json["leaderboard"]["players"]

team_list = []

for team in teams.keys():
    players = teams[team]
    for player in players:
        name = player.split()
        for i in range(0, field):
            try:
                if name[0] == score[i]["player_bio"]["first_name"]:
                    if name[1] == score[i]["player_bio"]["last_name"]:                          
                            row_dict = {"Team": team,
                                    "Player": player,
                                    'Today': int(score[i]["today"]),
                                    'Total' : score[i]["total"],
                                    'Thru': int(score[i]["thru"])}
                            team_list.append(row_dict)
                       
            except:
                print(i)

df = pd.DataFrame(team_list)
df = df.set_index("Team")
df = df[["Player","Today","Total","Thru"]]


#create team score

score_list = []
for team in teams.keys():
    score = 0
    count = -1 
    for i in list(df.index):
        count = count + 1
        if i == team:
            score = score + df.iloc[count][1]
    score_dict = {"Team": team, "Score": score}
    score_list.append(score_dict)        
    
df_score = pd.DataFrame(score_list)
df_score = df_score.set_index("Team")

df["Score"] = df_score

#clean scores

for i in range(0,len(df.index)):
    if i % len(list(teams[team1])) != 0:
        df.iloc[i, df.columns.get_loc('Score')] = ""
        
#drop the worst score

    else:
        df.iloc[i, df.columns.get_loc('Score')] = df.iloc[i][4] - df.iloc[i:i+len(list(teams[team1]))]["Today"].max()


df

102
104
81


,Player,Today,Total,Thru,Score
Team,,,,,
RBosshard,Louis Oosthuizen,2,0,18,0
RBosshard,Justin Rose,-2,-6,18,
RBosshard,Tiger Woods,0,-5,18,
RBosshard,Jordan Spieth,5,-4,18,
JKuo,Marc Leishman,5,5,18,
JKuo,Tony Finau,0,-4,18,4
JKuo,Tommy Fleetwood,2,-3,18,
JKuo,Brooks Koepka,-1,2,18,
JLo,Rickie Fowler,1,0,18,


In [20]:
#PGA Championship August 9, 2018

team1 = "RBosshard"
team1_list = {"Justin Thomas", "Jason Day", "Rickie Fowler", "Jon Rahm", "Francesco Molinari", "Paul Casey"}

team2 = "JKuo"
team2_list = {"Dustin Johnson", "Brooks Koepka", "Tommy Fleetwood", "Tony Finau", "Patrick Reed", "Hideki Matsuyama"}

team3 = "JLo"
team3_list = {"Rory McIlroy", "Justin Rose", "Jordan Spieth", "Henrik Stenson", "Xander Schauffele", "Tiger Woods"}


teams = {team1 : team1_list, team2 : team2_list, team3 : team3_list}

In [23]:
#PGA Championship August 9, 2018

PGA_base_url = "https://statdata.pgatour.com/r/" 

#Check PGA for current tournament

PGA_current= "https://statdata.pgatour.com/r/current/message.json"
r = requests.get(PGA_current)
r_json = r.json()
current_id = '033'
current_url = "https://statdata.pgatour.com/r/{0}".format(current_id)

#Current tournament's JSON

PGA_current_url = "https://statdata.pgatour.com/r/100/leaderboard-v2mini.json"
r = requests.get(PGA_current_url)
r_json = r.json()
current_tournament = r_json["debug"]["tournament_in_schedule_file_name"]

#Create Scoreboard

field = 156
score = r_json["leaderboard"]["players"]

team_list = []

for team in teams.keys():
    players = teams[team]
    for player in players:
        name = player.split()
        for i in range(0, field):
            try:
                if name[0] == score[i]["player_bio"]["first_name"]:
                    if name[1] == score[i]["player_bio"]["last_name"]:                          
                            row_dict = {"Team": team,
                                    "Player": player,
                                    'Today': int(score[i]["today"]),
                                    'Total' : score[i]["total"],
                                    'Thru': int(score[i]["thru"])}
                            team_list.append(row_dict)
                       
            except:
                print(i)

df = pd.DataFrame(team_list)
df = df.set_index("Team")
df = df[["Player","Today","Total","Thru"]]


#create team score

score_list = []
for team in teams.keys():
    score = 0
    count = -1 
    for i in list(df.index):
        count = count + 1
        if i == team:
            score = score + df.iloc[count][2]
    score_dict = {"Team": team, "Score": score}
    score_list.append(score_dict)        
    
df_score = pd.DataFrame(score_list)
df_score = df_score.set_index("Team")

df["Score"] = df_score

#clean scores

for i in range(0,len(df.index)):
    if i % len(list(teams[team1])) != 0:
        df.iloc[i, df.columns.get_loc('Score')] = ""
        
#drop the worst score

    else:
        df.iloc[i, df.columns.get_loc('Score')] = df.iloc[i][4] - df.iloc[i:i+len(list(teams[team1]))]["Total"].max()


df

102
81
104
87


,Player,Today,Total,Thru,Score
Team,,,,,
RBosshard,Rickie Fowler,1,0,18,-10
RBosshard,Jason Day,-3,-2,18,
RBosshard,Paul Casey,1,4,18,
RBosshard,Francesco Molinari,-2,-8,18,
JKuo,Patrick Reed,0,0,18,
JKuo,Tony Finau,0,-4,18,
JKuo,Tommy Fleetwood,2,-3,18,-7
JKuo,Brooks Koepka,-1,2,18,
JLo,Xander Schauffele,3,-6,18,
